In [111]:
import os
import re
import time
import urllib
import requests
from django.core.files import File
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
import json

In [105]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')

In [106]:
driver.get('https://www.dabangapp.com/room/5e992d31d9d3a15c88491696')

In [107]:
post_type = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/p/span')
post_type = post_type.get_attribute('innerText')

# 방 정보 설명
description = driver.find_elements_by_xpath("/html/body/div[1]/div/div[4]/div/div")
description = description[0].get_attribute("innerText")
description.replace("\n", "")

# 매물 형식
unrefined_salesform = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/div')
salesForm = unrefined_salesform[0].get_attribute("innerText")
salesForm = salesForm.replace('/', ' ')
salesForm = salesForm.replace('\n', '')
salesForm = salesForm.split()
salesType = salesForm[0]  # sales type


In [108]:
post_type

'아파트'

In [109]:
salesType

'월세'

In [110]:

if post_type == "아파트":
    if salesType == "매매":
        address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/p')
    else:
        address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[5]/div/p')
else:
    address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[4]/div/p')
if not address:
    address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div/div/p')

try:
    address = address[0].get_attribute('innerText')
    if '※' in address:
        address = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[4]/div/p')

        address = address.get_attribute('innerText')
    print('address >>>>>>>>>>>>', address)
except NoSuchElementException:
    address = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[5]/div/p')
    address = address.get_attribute('innerText')

print('address', address)


address >>>>>>>>>>>> 서울시 성동구 성수동1가 718
address 서울시 성동구 성수동1가 718


# 카카오 로컬

In [112]:
url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'

In [113]:
res = requests.get(url, headers={'Authorization':'KakaoAK c5fe0d7cf28e7714d8f01200f604a03b'})

In [114]:
res.headers

{'Server': 'nginx', 'Date': 'Sat, 25 Apr 2020 17:35:56 GMT', 'Content-Type': 'application/json;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-Request-Id': '3867e0a0-871b-11ea-ae56-a3ccf9d6ecc6', 'Content-Encoding': 'gzip', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, OPTIONS', 'Access-Control-Allow-Headers': 'Authorization, KA, Origin, X-Requested-With, Content-Type, Accept'}

In [115]:
res.reason

'OK'

In [116]:
res.url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%EC%8B%9C%20%EC%84%B1%EB%8F%99%EA%B5%AC%20%EC%84%B1%EC%88%98%EB%8F%991%EA%B0%80%20718'

In [117]:
res.text

'{"documents":[{"address":{"address_name":"서울 성동구 성수동1가 718","b_code":"1120011400","h_code":"1120065000","main_adderss_no":"","main_address_no":"718","mountain_yn":"N","region_1depth_name":"서울","region_2depth_name":"성동구","region_3depth_h_name":"성수1가1동","region_3depth_name":"성수동1가","sub_adderss_no":"","sub_address_no":"","x":"127.045045893181","y":"37.5388239825147","zip_code":""},"address_name":"서울 성동구 성수동1가 718","address_type":"REGION_ADDR","road_address":{"address_name":"서울 성동구 왕십리로 16","building_name":"트리마제","main_building_no":"16","region_1depth_name":"서울","region_2depth_name":"성동구","region_3depth_name":"성수동1가","road_name":"왕십리로","sub_building_no":"","undergroun_yn":"","underground_yn":"N","x":"127.044671518265","y":"37.5392578661678","zone_no":"04773"},"x":"127.045045893181","y":"37.5388239825147"}],"meta":{"is_end":true,"pageable_count":1,"total_count":1}}\n'

In [118]:
str_data =res.text

In [119]:
json_data = json.loads(str_data)

In [124]:
lat = json_data['documents'][0]['x']

In [125]:
lng = json_data['documents'][0]['y']

In [126]:
lat

'127.045045893181'

In [127]:
lng

'37.5388239825147'